In [2]:
import pandas as pd
import statistics
import os
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, scale
from sklearn.utils import shuffle
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import scale, StandardScaler
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import precision_score
from sklearn.metrics import auc
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score

import matplotlib.pyplot as plt
import matplotlib
import numbers
import decimal
import sys
import ast
import numpy as np
from scipy import interp
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold


resultsList=[]
resultsList.append(["Classifier Name","Accuracy", "Mean Accuracy", "Standard Deviation of Accuracy","Precision", "Recall","Area Under Curve", "F1 Score"])


FILE_NAME="Input_df_4.xlsx"

data=pd.read_excel(FILE_NAME,index_col=False)
# data.columns = ['Country of the Client', 'Industrial Sector of the Client','How was the Deal Identified', 'Type of the Client', 'Relationship with the Client', 'Value (M$)', 'Outcome']

my_cols = list(data.columns)
my_cols.pop()
my_cols.pop()
data=pd.get_dummies(data,columns=my_cols)

Features=data.drop(["Outcome"],axis=1)

X_train, X_test, y_train, y_test = train_test_split(Features, data["Outcome"],train_size=0.8,test_size=0.2,stratify=data["Outcome"], random_state=42)

param_grid = {"C":10.**np.arange(-1,1)}   
#Note the use of 10. not 10 because even in python 3, 
#the exponential power is integer unless the base is float, so 10**-3 is actually zero 
# (and that wouldn't work for the C parameter of logistic regression)

grid_search = GridSearchCV(LogisticRegression(solver='lbfgs'), param_grid, cv=10)
grid_search.fit(X_train, y_train)
grid_search.predict(X_test)

# print("Logistic Regression Results")
# print(grid_search.score(X_test, y_test))
# print("\n")
LogisticRegressionResults=grid_search.score(X_test, y_test)
#print(grid_search.best_params_)
y_score=  grid_search.predict(X_test)
myClassifier="Logistic Regression"
#Calculate Accuracy, Precision, Recall, AUC, F1_score
myAccuracy=grid_search.score(X_test, y_test)
fpr,tpr,thresholds=roc_curve(y_test,y_score)
myAUC = auc(fpr, tpr)
myRecall=recall_score(y_test,y_score,pos_label=1,average='binary')
myPrecision=precision_score(y_test,y_score,pos_label=1,average='binary')
myF1Score=f1_score(y_test,y_score,pos_label=1,average='binary')

scores=cross_val_score(grid_search,Features,data["Outcome"],cv=10)
#print(scores.mean())
#print(scores.std())
resultsList.append([myClassifier,myAccuracy,scores.mean(),scores.std(),myPrecision,myRecall,myAUC,myF1Score])

# print("Logistic Regression Results", ",",LogisticRegressionResults)

est = grid_search.best_estimator_
varNames = list(Features.columns)
for i in range(len(varNames)):
    print(varNames[i] + ":" + str(est.coef_[0, i]))

coef = est.coef_[0]

Value (M$):0.02276291094016744
Country of the Client_0:-0.030496931240874003
Country of the Client_1:0.03690184939403138
Industrial Sector of the Client_0:-0.218831501028789
Industrial Sector of the Client_1:-0.05548397254835568
Industrial Sector of the Client_2:0.04930409006498305
Industrial Sector of the Client_3:-0.02330868431051232
Industrial Sector of the Client_4:0.02811868354510513
Industrial Sector of the Client_5:0.13144448349077906
Industrial Sector of the Client_6:-0.4389898867623361
Industrial Sector of the Client_7:0.534151705702271
How was the Deal Identified_0:-0.08666898575782872
How was the Deal Identified_1:0.15921745253723496
How was the Deal Identified_2:-0.1932444159861148
How was the Deal Identified_3:-0.17253307405605164
How was the Deal Identified_4:0.29963394141592625
Type of the Client_0:0.16771658343605764
Type of the Client_1:-0.15141172564834987
Type of the Client_2:0.005788829647146701
Type of the Client_3:0.22643448916790418
Type of the Client_4:0.3260922

In [3]:
grid_search.get_params()

{'cv': 10,
 'error_score': 'raise-deprecating',
 'estimator__C': 1.0,
 'estimator__class_weight': None,
 'estimator__dual': False,
 'estimator__fit_intercept': True,
 'estimator__intercept_scaling': 1,
 'estimator__l1_ratio': None,
 'estimator__max_iter': 100,
 'estimator__multi_class': 'warn',
 'estimator__n_jobs': None,
 'estimator__penalty': 'l2',
 'estimator__random_state': None,
 'estimator__solver': 'lbfgs',
 'estimator__tol': 0.0001,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'estimator': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='warn', n_jobs=None, penalty='l2',
                    random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 'iid': 'warn',
 'n_jobs': None,
 'param_grid': {'C': array([0.1, 1. ])},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': False,
 '

In [4]:
grid_search.best_estimator_

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [6]:
est.coef_

array([[ 0.02276291, -0.03049693,  0.03690185, -0.2188315 , -0.05548397,
         0.04930409, -0.02330868,  0.02811868,  0.13144448, -0.43898989,
         0.53415171, -0.08666899,  0.15921745, -0.19324442, -0.17253307,
         0.29963394,  0.16771658, -0.15141173,  0.00578883,  0.22643449,
         0.32609226, -0.41567131, -0.30636518, -0.31114268,  0.46496365,
        -0.45350784, -0.33802347, -0.13194042,  0.22368428,  0.70619237]])